# SQLAlchemy och Pandas

För att koppla ihop SQL-databasen med Python behöver vi använda ett bibliotek som heter SQLAlchemy. Dessutom behövs ett bibliotek som heter pyodbc för att hantera kopplingen mellan SQLAlchemy och SQL-servern.

Följande kodexempel bör fungera för att skapa en koppling mellan SQL-servern och en Jupyter Notebook för att utföra en query och läsa in resultatet som en Pandas `DataFrame` för vidare analys och visualisering.

# Teoretiska frågor
1. Beskriv kort hur en relationsdatabas fungerar.
Relationsdatabaser fungerar med hjälp av tabeller för att organisera data. Eftersom det är data i olika tabeller så kan man sedan koppla tabeller med varandra vilket ger en relation mellan datan som finns i olika tabeller.
2. Vad menas med “CRUD”-flödet?
CRUD står för Create, Read, Update och Delete och är väldigt grundläggande i vad man kan göra i en databas. Ska man exempelvis gå igenom en databas för första gången så kan man använda sig av delar i CRUD som exempel, läsa igenom, uppdatera, skapa och ta bort viss data. 
3. Beskriv kort vad en “left join” och en “inner join” är. Varför använder man det?
Man använder olika joins för att kombinera två olika tabeller. En inner join använder man för att kombinera två tabeller med en likhet och att det har gemmensamma förutsättningar. En left join använder man om man vill spara alla ha kvar alla rader från den vänstra tabellen men inte den andra, dvs även om den inte har likadana rader på andra tabellen så stannar alla rader på den vänstra tabellen.
4. Beskriv kort vad indexering i SQL innebär.
När man indexerar i SQL så gör man det för att skapa en sorts datastruktur alltså en index eller som ett register. Inderxering gör så att du kan hitta liknande rader i tabellerna utan att behöva gå igenom hela tabellen. Det kan användas om tabellen har många rader för att hitta snabbare.  
5. Beskriv kort vad en vy i SQL är. 
En vy används för att förenkla tuffa frågor samt en förbättrad säkerhet på datan som visas i en vy. Du kan också använda en vy när du ska presentera en viss data på ett säkert sätt.
6. Beskriv kort vad en lagrad procedur i SQL är.
En lagrad procedur i SQL kan vara querys dvs kommandon som sparas i databasen som sedan kan användas för att för att få en viss funktion. Har man återkommande arbetsuppgifter så är en lagrad procedur bra för att kunna snabbt få upp samma kommando igen för att lösa arbetsuppgiften på nytt. 

In [ ]:
# Installera bibliotek. Om du använder Anaconda är de antagligen redan installerade och du bör inte behöva inte köra den här koden.

! pip install sqlalchemy pyodbc pandas jupyter

In [2]:
# Importer

from sqlalchemy.engine import create_engine, URL
import pandas as pd

In [3]:

SERVER = 'localhost'  # localhost innebär att du ansluter till en server på din egen dator
DATABASE = 'AdventureWorks2022'

connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;Encrypt=no'

connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})

engine = create_engine(connection_url)  # engine är anslutningsobjektet vi använder för att kommunicera med databasen.

# Query ett - Här så väljer jag produkt namn samt försäljningen av produkt

I denna query så använder jag P.ProductID för att få ut produkt id från tabellen Production.Product.
I samband med det så gör jag en P.Name för att ge ett alias för läsbarhetens skull och använder “AS” för att göra det. Alltså P.Name AS ProductName 
Sedan använder jag SUM som då betyder summation och summerar i mitt fall då antalet sålda produkter från Sales.SalesOrderDetail (SOD). Igen använder jag “AS” för att göra det mer läsbart till TotalQuantitySold
Samma princip gjorde jag i nästa del av koden för att göra det lätt att förstå hur mycket i antal som sålts vilket fick heta TotalSalesRevenue.

FROM o JOIN
I min FROM statement använder jag Production.Product med en förkortning som P.
SEdan använder jag JOIN för koppla ihop tabellen Sales.SalesOrderDetail och förkortar den med SOD.
För att sen få matchande rader som jag hade sett var ProductID så använde jag ON statement för att säkerställa att just ProductID skulle inkluderas.

GROUP BY o ORDER BY
Eftersom jag använder SUM alltså summeringsfunktion så är det viktigt att använda gruppering då jag vill få alla totaler. För att sen separera använder jag GROUP BY för att separera produkter med sina ID:n med P.ProductID
och samma för namnen som är då P.Name. 

Till sist använder jag ORDER BY på TotalSalesRevenue för att få resultaten på de som har sålts mest och som visar i fallande ordning där jag använder DESC som står för descending. 


In [4]:
# Skriv en query. Använd trippla citationstecken runt textsträngen för att kunna ha radbrytningar i den.

query = """
SELECT 
P.ProductID, P.Name AS ProductName, SUM(SOD.OrderQty) AS TotalQuantitySold, SUM(SOD.LineTotal) AS TotalSalesRevenue
FROM
Production.Product P
JOIN
Sales.SalesOrderDetail SOD
ON
P.ProductID = SOD.ProductID
GROUP BY
P.ProductID, P.Name
ORDER BY
TotalSalesRevenue DESC;"""

# Skapa en DataFrame med resultaten av queryn.
products_df = pd.read_sql(query, engine)

In [ ]:
products_df

# Konfidensintervall - Produkter

In [ ]:
import numpy as np
from scipy.stats import t, sem

försäljning = products_df['TotalSalesRevenue'].values 
konfidens_intervall = 0.95

medelvärde = försäljning.mean()
standardfel = sem(försäljning)  

t_värde = t.ppf((1 + konfidens_intervall) / 2, df=len(försäljning)-1)
marginal_fel = t_värde * standardfel

lower = medelvärde - marginal_fel
upper = medelvärde + marginal_fel

print(f"Medelvärde av försäljning: {medelvärde:.2f}")
print(f"95% Konfidensintervall: ({lower:.2f}, {upper:.2f})")

# Query två - Nu tar vi reda på vilken månad som sålt mest

Här så börjar jag med att dela upp YEAR som jag hämtar ifrån Sales.SalesOrderHeader (SOH) vilket hämtar åren från kolumnen OrderDate. Jag döper den till Year.
Samma procedur gör jag med MONTH för att få fram månader från OrderDate.
Sedan summerar jag kvantiteter från (SOD) som heter OrderQty för att döpa den till TotalQuantitySold.
För att få den totala försäljningen så summerar jag från (SOD) som heter LineTotal  och döper den till TotalSalesRevenue.


In [7]:
query = """
SELECT
YEAR(SOH.OrderDate) AS Year, MONTH(SOH.OrderDate) AS Month, SUM(SOD.OrderQty) AS TotalQuantitySold, SUM(SOD.LineTotal) AS TotalSalesRevenue
FROM
Sales.SalesOrderHeader SOH
JOIN
Sales.SalesOrderDetail SOD
ON
SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY
YEAR(SOH.OrderDate), MONTH(SOH.OrderDate)
ORDER BY
TotalSalesRevenue DESC;"""


datum = pd.read_sql(query, engine)

In [ ]:
datum

# Konfidensintervall - Datum

In [ ]:
import numpy as np
from scipy.stats import t, sem

försäljning_enheter = datum['Month'].values 
konfidens_intervall = 0.95

medelvärde = försäljning_enheter.mean()
standardfel = sem(försäljning_enheter)  

t_värde = t.ppf((1 + konfidens_intervall) / 2, df=len(försäljning_enheter)-1)
marginal_fel = t_värde * standardfel

lower = medelvärde - marginal_fel
upper = medelvärde + marginal_fel

print(f"Medelvärde av försäljning av enheter: {medelvärde:.2f}")
print(f"95% Konfidensintervall: ({lower:.2f}, {upper:.2f})")

# Query tre - Eftersom vi vet nu att år 2014 samt månad 3 var best säljade så filtrerar vi produkter för just de datum


Den ända skillnaden här är att nu eftersom jag vet att den är 2014 och månad 3 som sålt mest så stoppade jag in dom värdena i min WHERE statement för bara inkludera just den månaden från de året. Och det var intressant att se hur en moutainbike va så populärt. 

In [10]:
query = """
SELECT
P.ProductID,  P.Name AS ProductName, SUM(SOD.OrderQty) AS TotalQuantitySold, SUM(SOD.LineTotal) AS TotalSalesRevenue
FROM
Production.Product P
JOIN
Sales.SalesOrderDetail SOD
ON
P.ProductID = SOD.ProductID
JOIN
Sales.SalesOrderHeader SOH
ON
SOD.SalesOrderID = SOH.SalesOrderID
WHERE
YEAR(SOH.OrderDate) = 2014 AND MONTH(SOH.OrderDate) = 3
GROUP BY
P.ProductID, P.Name
ORDER BY
TotalSalesRevenue DESC;"""


mars_månad = pd.read_sql(query, engine)

In [ ]:
mars_månad

# Konfidensintervall - 2014 månad 3

In [ ]:
import numpy as np
from scipy.stats import t, sem

försäljning = mars_månad['TotalSalesRevenue'].values 
konfidens_intervall = 0.95

medelvärde = försäljning.mean()
standardfel = sem(försäljning)  

t_värde = t.ppf((1 + konfidens_intervall) / 2, df=len(försäljning)-1)
marginal_fel = t_värde * standardfel

lower = medelvärde - marginal_fel
upper = medelvärde + marginal_fel

print(f"Medelvärde av försäljning: {medelvärde:.2f}")
print(f"95% Konfidensintervall: ({lower:.2f}, {upper:.2f})")

# Query fyra - Top 10
Yttligare så valde jag att titta på top 10 varor som sålts mest. Där jag räknar ut konfidensintervallet samt medelvärde.

In [13]:
query = """ 
SELECT TOP 10
    P.ProductID,  
    P.Name AS ProductName, 
    SUM(SOD.OrderQty) AS TotalQuantitySold, 
    SUM(SOD.LineTotal) AS TotalSalesRevenue
FROM
    Production.Product P
JOIN
    Sales.SalesOrderDetail SOD
    ON P.ProductID = SOD.ProductID
JOIN
    Sales.SalesOrderHeader SOH
    ON SOD.SalesOrderID = SOH.SalesOrderID
WHERE
    YEAR(SOH.OrderDate) = 2014 
    AND MONTH(SOH.OrderDate) = 3
GROUP BY
    P.ProductID, 
    P.Name
ORDER BY
    TotalSalesRevenue DESC;"""

tio_top = pd.read_sql(query, engine)

In [ ]:
tio_top

# Konfidensintervall - TOP 10

In [ ]:
import numpy as np
from scipy.stats import t, sem

försäljning = tio_top['TotalSalesRevenue'].values 
konfidens_intervall = 0.95

medelvärde = försäljning.mean()
standardfel = sem(försäljning)  

t_värde = t.ppf((1 + konfidens_intervall) / 2, df=len(försäljning)-1)
marginal_fel = t_värde * standardfel

lower = medelvärde - marginal_fel
upper = medelvärde + marginal_fel

print(f"Medelvärde av försäljning: {medelvärde:.2f}")
print(f"95% Konfidensintervall: ({lower:.2f}, {upper:.2f})")

# Kort sammanfattning

Query ett - Så kollar jag upp alla produkter som företaget har, och kollar på hur mycket som sålts som en helhet av alla produkter med en 95% kofidensintervall.
Query två - Här ser vi datum för best sålda datum. Med en 95% konfidensintervall så ser vi hur mycket av produkter som sålts under månader. 
Query tre - Nu filtrerar jag till just den bäst säljande datum vilket är 2014 mars där vi kollar på konfidensintervall på försäljningsintäkter. Vi kan se att för månaden 3 år 2014 så har vi ungeräflig medelvärde i försäljningen på 40000 men att vi är 95% säkra på att försäljningen skulle ungefärlig vara mellan 30000 till 50000.
Query fyra - Tillsist filtrerar jag varför just dessa produkter som sålts bäst och räknar slutligen ut konfidensintervall på just de 10 produkter som sålt bäst i Mars 2014.

# Rekomendationer
Som rekomendation kan vi se i min tredje query samt konfidensintervallet att det är vår/sommar och många väljer att köpa moutainbikes för att snön kan smälta och det blir vår/sommar väder. Jag hade gjort kampanjer som vår/sommar rea för cyklar samt cykel accesoarer då vi ser att det finns en tendens att köparna köper gärna under varmare väder. Men det behöver inte heller vara så, som test och rekomendation hade jag ändå gjort en vår rea. 